In [1]:
from tensorflow.keras.models import Model
import tensorflow as tf
from model import lstm_unet
import keras.backend as K
from helpers import f1
import warnings
import numpy as np
import matplotlib.pyplot as plt
from data_loader import DataGenerator
warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
   tf.config.set_visible_devices(gpus[0], 'GPU')
   tf.config.experimental.set_memory_growth(gpus[0], True)

2025-08-17 18:04:33.253897: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-17 18:04:33.291058: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-17 18:04:33.291094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-17 18:04:33.292438: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-17 18:04:33.299174: I tensorflow/core/platform/cpu_feature_guar

In [2]:
model = lstm_unet()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1])
model.summary()

2025-08-17 18:04:35.437504: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46672 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:21:00.0, compute capability: 8.9


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 15, 256, 256, 3)]    0         []                            
                                                                                                  
 time_distributed (TimeDist  (None, 15, 256, 256, 16)     448       ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 time_distributed_1 (TimeDi  (None, 15, 256, 256, 16)     64        ['time_distributed[0][0]']    
 stributed)                                                                                       
                                                                                              

In [3]:
train_gen = DataGenerator(
    image_dir="../Unet/data_v3_processed/train/images/images",
    mask_dir="../Unet/data_v3_processed/train/masks/masks",
    batch_size=8,
    time_steps=15,
    img_size=(256,256),
    normalize_images=True,
    normalize_masks=False,
    infinite=True,
)

In [4]:
val_gen = DataGenerator(
    image_dir="../Unet/data_v3_processed/val/images/images",
    mask_dir="../Unet/data_v3_processed/val/masks/masks",
    batch_size=8,
    time_steps=15,
    img_size=(256,256),
    normalize_images=True,
    normalize_masks=False,
    infinite=True
)

In [5]:
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=1,
                    steps_per_epoch=3000,
                    validation_steps=13)

2025-08-17 18:04:47.498104: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-08-17 18:04:53.474043: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8ecc36b370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-17 18:04:53.474090: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2025-08-17 18:04:53.478666: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1755453893.578734  133398 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3000/3000 [==============================] - 1291s 420ms/step - loss: 0.0357 - f1: 0.7499 - val_loss: 0.0088 - val_f1: 0.7053


In [7]:
model.save("lstm_unet_model.h5")